In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.precision', 15)  # Increase decimal precision
pd.set_option('display.width', 150)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

# Phương pháp Lặp đơn cho Hệ phi tuyến

Điều kiện hội tụ: $||G|| \leq q < 1$ --> đảm bảo nghiệm duy nhất

Đánh giá sai số:

* Tiên nghiệm: * Hậu nghiệm: $||x_n - x^*|| \leq \dfrac{q^n}{1-q} ||x_1-x_0||$

* Hậu nghiệm: $||x_n - x^*|| \leq \dfrac{q}{1-q} ||x_n-x_{n-1}||$



# Thuật toán

1. **Khởi tạo**: 

- Chọn giá trị ban đầu $x_0$ và $y_0$.

- Tạo phương trình lặp $x = g_1(x)$ tương đương với phương trình $f_1(x, y) = 0$.

  Tạo phương trình lặp $y = g_2(y)$ tương đương với phương trình $f_2(x, y) = 0$.

2. **Lặp**: Thực hiện các bước sau

- Tính giá trị mới của $x$ và $y$:

  $x_{k+1} = g_1(x_k, y_k)$, $\quad\quad$ $y_{k+1} = g_2(x_k, y_k)$
      
- Kiểm tra điều kiện hội tụ: Dừng nếu đạt được độ chính xác mong muốn, hoặc đạt đến số lần lặp tối đa.

3. **Kết quả**: Nếu $\exists \lim\limits_{k \to \infty} x_k = \alpha$ và $g_1(x)$ liên tục thì $\alpha = g_1(\alpha)$. Tương tự $\beta = g_2(\beta)$

# Áp dụng

## 1. Chuẩn 1 (chuẩn cột)

$||A||_{(1)} = \max\limits_{1\leq j \leq n} \sum\limits_{i=1}^{m} |a_{ij}|$



### 1.1. Tiên nghiệm

In [ ]:
def fixed_point_iteration(initial_values, tol=1e-6, max_iter=100, *funcs):
    n = len(initial_values)
    values = np.array(initial_values)
    results = []

    for i in range(max_iter):
        new_values = np.array([funcs[j](*values) for j in range(n)])
        
        results.append([i+1] + new_values.tolist())
        
        if np.all(np.abs(new_values - values) < tol):
            break
        
        values = new_values
    
    columns = ['Iteration'] + [f'x{j+1}' for j in range(n)]
    df = pd.DataFrame(results, columns=columns)
    print(df)
    
    return values


### 1.2. Hậu nghiệm


In [2]:
def fixed_point_iteration_v1(initial_values, q, eps, *funcs):
    n = len(initial_values)
    values = np.array(initial_values)
    results = [[0] + initial_values]; i=0;

    new_eps = eps * (1-q)/ q
    print(f"new_eps: {new_eps}")

    while True:
        new_values = np.array([funcs[j](*values) for j in range(n)])
        
        # Calculate the differences
        diffs = np.abs(new_values - values)
        diff_sum = np.sum(diffs)
        
        # Append the iteration number, new values, differences, and diff_sum to the results list
        results.append([i+1] + new_values.tolist() + diffs.tolist() + [diff_sum])
        
        # Check for convergence
        if diff_sum < new_eps:
            break
        
        values = new_values
        i += 1  # Increment the iteration counter
    
    columns = ['Iteration'] + [f'x{j+1}' for j in range(n)] + [f'diff_x{j+1}' for j in range(n)] + ['diff_sum']
    df = pd.DataFrame(results, columns=columns)
    print(df.to_string(index=False))  # Print the DataFrame without the index
    
    return values

In [3]:
def g1(x, y, z):
    return 1/3 * (np.cos(y*z) + 0.5)

def g2(x, y, z):
    return 1/9 * np.sqrt(x**2 + np.sin(z) + 1.06) - 0.1

def g3(x, y, z):
    return -1/20 * np.exp(-x*y) - 9.1389/20

# Store g functions
def store_g_funcs(*g_funcs):
    return g_funcs

g_funcs = store_g_funcs(g1, g2, g3)

# Initial guess
initial_values = [1, 0, 0.5]
eps = 0.5 * 1e-9
q = 0.280 * 3

# Perform fixed-point iteration
solution = fixed_point_iteration_v1(initial_values, q, eps, *g_funcs)
print("Approximate solution:", solution)

new_eps: 9.52380952380952e-11
 Iteration                x1                x2                 x3           diff_x1           diff_x2           diff_x3          diff_sum
         0 1.000000000000000 0.000000000000000  0.500000000000000               NaN               NaN               NaN               NaN
         1 0.500000000000000 0.077061945587866 -0.506945000000000 0.500000000000000 0.077061945587866 1.006945000000000 1.584006945587866
         2 0.499745671434312 0.000890556858707 -0.505055095109852 0.000254328565688 0.076171388729159 0.001889904890148 0.078315622184994
         3 0.499999966282951 0.000976105148869 -0.506922752354281 0.000254294848639 0.000085548290162 0.001867657244429 0.002207500383229
         4 0.499999959193860 0.000891744791128 -0.506920603326837 0.000000007089091 0.000084360357741 0.000002149027444 0.000086516474276
         5 0.499999965942733 0.000891859305896 -0.506922711351357 0.000000006748873 0.000000114514768 0.000002108024519 0.000002229288160
    